## HW5Q2

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivychen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
with open('/project2/cmsc25025/wikipedia/wiki-text.txt', 'r') as f:
    wiki = f.read()
    words = "".join(wiki)
    words = words.split(" ")

In [6]:
len(words)

124301827

In [7]:
# find the word frequency excluding the stop words
fdist = FreqDist()
for word in words:
    if word.lower() not in stop_words:
        fdist[word.lower()] += 1

In [8]:
# remove words that appear less than n times 
def remove_words(n):
    V = set()
    for word in fdist:
        if fdist[word] >= n:
            V.add(word)
    return list(V)

In [9]:
# expriment with n for several times, if n = 400, we 
# have the number of words in the vocabulary roughly 
# equal to 15,000 (15, 348). V: key: word, value: ind
V = remove_words(400)

In [10]:
# select a subset of texts to expriment first 
# remove empty string
words = [words[i] for i in range(len(words)) if words[i] != ""]
# sub_words = words[:100000]

In [13]:
# Calculate PMI 
# Define S^P and N^p(w,c) with a window of size 5 
import json
from collections import defaultdict
def create_sp(words):
    # create a default dictionary dict: word- context- frequency
    word_context = defaultdict(lambda: defaultdict(int))
    length = len(words)
    for i in range(length):
        # track 
        if i%1000000 == 0:
            print(i//1000000) 
            
        if i%3000000 == 0:
            with open('W{}'.format(i), 'w') as f:
                json.dump(word_context, f)
            
        word = words[i]
        if word in V:
            if i <= 4:
                context = [words[j] for j in range(-(5-i), (i+6)) if j !=i]
            elif i >= length-5:
                context = [words[j] for j in range(-(5-i), length) if j !=i]
                context += [words[j] for j in range(i+6-length)]
            else:
                context = [words[i+j] for j in range(-5, 6) if j !=0]
            for c in context:
                word_context[word][c] += 1
    return word_context

In [117]:
wc = create_sp(words)

In [94]:
#import json
#with open('W', 'w') as f:
#    json.dump(wc, f)

In [95]:
with open('W','r') as f:
    wc1 = json.load(f)

In [18]:
# calculate N^p and N^c
def compute_np(word_context):
    wset, cset = defaultdict(int), defaultdict(int)
    for w, d in word_context.items():
        wset[w] = sum(d.values())
        for c, f in d.items():
            cset[c] += f
    return wset, cset       

In [22]:
def create_w(vocab, word_con_pair):
    n = len(vocab)
    M = np.zeros([n,n])
    wset, cset = compute_np(word_con_pair)
    for i in range(n):
        if i%1000 == 0:
            print(i//1000)         
        for j in range(n):
            w, c = vocab[i], vocab[j]
            nij = word_con_pair[w][c]
            ni = wset[w] 
            nj = cset[c]
            M[i, j] = np.log( (nij+1)/(ni*nj) ) 
    return M

In [118]:
M = create_w(V, wc)

In [25]:
np.save('M', M)

In [26]:
### create word embeddings 
import scipy
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
u, s, vt = svds(csr_matrix(M), k=50)

In [103]:
W = u.dot(np.diag(np.sqrt(s)))

In [35]:
# serialization and store
with open('Wmat.pickle', 'wb') as handle:
    pickle.dump(W, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
# with open('Wmat.pickle', 'rb') as handle:
#    b = pickle.load(handle)
# print(W == b)

[ True  True  True ...  True  True  True]


In [71]:
# find the 5 closest words in the embedding space: physics, republican, einstein, algebra, fish
print("physics: ", V.index('physics'))
print("republican: ", V.index('republican'))
print("einstein: ", V.index('einstein'))
print("algebra: ", V.index('algebra'))
print("fish: ", V.index('fish'))

physics:  7704
republican:  7043
einstein:  9162
algebra:  3429
fish:  14517


In [109]:
from numpy.linalg import norm as norm
def fiveClosest(word, V, W):
    wind = V.index(word)
    ind = np.argsort([norm(W[i] - W[wind]) for i in range(len(W))])[1:6]
    return [V[i] for i in ind]

In [119]:
fiveClosest('physics', V, W)

['mechanics', 'chemistry', 'quantum', 'mathematics', 'theoretical']

In [120]:
fiveClosest('republican', V, W)

['senator', 'democrat', 'democrats', 'representative', 'candidate']

In [121]:
fiveClosest('einstein', V, W)

['relativity', 'paradox', 'planck', 'maxwell', 'leibniz']

In [122]:
fiveClosest('algebra', V, W)

['algebraic', 'finite', 'theorem', 'topology', 'calculus']

In [110]:
fiveClosest('fish', V, W)

['fruit', 'meat', 'eggs', 'cattle', 'wild']

In [126]:
# find analogies v= v_democrat - v_repub + v_conserv
def findAnalogies(w1, w2, w3, V, W):
    wi1, wi2, wi3 = V.index(w1), V.index(w2), V.index(w3)
    ana = W[wi1] - W[wi2] + W[wi3] 
    return [V[j] for j in np.argsort([norm(W[i]-ana) for i in range(len(W))])[:5]]

In [127]:
findAnalogies('republican', 'democrat', 'conservative', V, W)

['conservative', 'liberal', 'opposition', 'democratic', 'leaders']